In [ ]:
import networkx as nx
import networkx.algorithms.community as nxcom
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from ipywidgets import interact, IntRangeSlider
%matplotlib inline
from IPython.display import display, clear_output
import pickle
import numpy as np

In [ ]:
with open('pickle/treasury.p', 'rb') as f: 
    G = pickle.load(f)

In [ ]:
H = nx.DiGraph.to_undirected(G)

In [ ]:
minyear = min(nx.get_edge_attributes(G, "min_date").values())
maxyear = max(nx.get_edge_attributes(G, "max_date").values())

In [ ]:
def discrete_cmap(N, base_cmap=None):
    """Create an N-bin discrete colormap from the specified input map
    https://gist.github.com/jakevdp/91077b0cae40f8f8244a
    by Jake VanderPlas"""
    base = plt.cm.get_cmap(base_cmap)
    color_list = base(np.linspace(0, 1, N))
    return color_list

In [ ]:
@interact(years = IntRangeSlider(value = [minyear, maxyear], min = minyear, max=maxyear))
def restrict_by_years(years):
    s_edges = [(s, t) for s,t,a in H.edges(data=True) if years[0] < a['min_date'] and years[1] > a['max_date']]
    graph = H.edge_subgraph(s_edges)
    k_clique_com = sorted(nxcom.k_clique_communities(graph, 4), key=len, reverse=True)
    # Count the communities
    N = len(k_clique_com)
    colmap = discrete_cmap(N+2, "jet")
    colmap[0] = [0, 0, 0, 1]  # 0 = black
    print(f"The graph has {N} clique communities.")
    print(f"Edges: {graph.number_of_edges()}")
    nx.set_node_attributes(H, {0}, "cliques") # set {0} as default
    for c, v_c in enumerate(k_clique_com):
        for v in v_c:
            if min(graph.nodes[v]['cliques']) > 0:  # node is already in a clique
                graph.nodes[v]['cliques'].add(c+1) # add clique no. to the set
            else:
                graph.nodes[v]['cliques'] = {c+1}
    nx.set_edge_attributes(H, 0, "clique")  # default is 0
    for v, w, in H.edges:
        c = H.nodes[v]['cliques'].intersection(H.nodes[w]['cliques']) # do the two nodes share a community?
        if c:
            # Internal edge, mark with community
            H.edges[v, w]['clique'] = min(c)
    nodes = [x for x, y in graph.nodes(data=True) if min(y["cliques"]) > 0 or y["degree"] > 8] # select nodes
    I = graph.subgraph(nodes)
    node_color = [colmap[min(I.nodes[v]['cliques'])] if len(I.nodes[v]['cliques'])==1 else colmap[N+1] for v in I.nodes]
    edge_color = [colmap[I.edges[v, w]['clique']] for v, w in I.edges]
    labels = nx.get_node_attributes(I, 'name')
    label_pos = {}
    pos = nx.circular_layout(I)
    for k, v in pos.items():   # shift position of label slightly away from the node
        label_pos[k] = (v[0], v[1]+0.03)
    fig = plt.figure(figsize=(15, 12))
    plt.gca().margins(0.15, 0.15) # provide margins to prevent labels from going over the edge
    nx.draw(
        I,
        pos=pos,
        node_color=node_color,
        edge_color=edge_color, 
        #node_size = node_size, 
        with_labels = False,
        cmap = colmap, 
        width = 3, 
        alpha = .65
        #cmap= plt.cm.jet
        )
    nx.draw_networkx_labels(I, label_pos, labels, font_size=16)
    # the following are fake "lines" used to create the legend.
    legend_elements = [Line2D([1], [0], marker='o', color='w', label=str(i+1),
                          markerfacecolor=colmap[i+1], markersize=15) for i, c in enumerate(k_clique_com)]
    no_com = [Line2D([0], [0], marker='o', color='w', label="None",
                          markerfacecolor=colmap[0], markersize=15)]
    mult_com = [Line2D([0], [0], marker='o', color='w', label="Multiple",
                          markerfacecolor=colmap[N+1], markersize=15)]
    legend_elements = no_com + legend_elements + mult_com
    fig.legend(handles=legend_elements)
    plt.show()